In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
data = 'C:\\Users\\Admin\\Desktop\\Herbal-Identification\\Plant-Dataset\\'
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_ds = ImageFolder(data + 'New Plant-Dataset/train', transform=transform)

# Load the model
model = models.resnet34(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(test_ds.classes))  # Adjust the final layer to the number of classes
model.load_state_dict(torch.load('herbalIdentification-resnet34.pth'))
model = model.to(device)
model.eval()  # Set the model to evaluation mode

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def predict_image(img, model, class_names):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    probs = F.softmax(yb, dim=1)  # Apply softmax to get probabilities
    confidence, preds = torch.max(probs, dim=1)
    return class_names[preds[0].item()], confidence.item() * 100  # Return the class name and confidence percentage

# Test with a sample image
img, label = test_ds[25]
plt.imshow(img.permute(1, 2, 0))
predicted_class, confidence = predict_image(img, model, test_ds.classes)
print(f'Label: {test_ds.classes[label]}, Predicted: {predicted_class}, Confidence: {confidence:.2f}%')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Admin\\Desktop\\Herbal-Identification\\Plant-Dataset\\New Plant-Dataset/train'